Geothermal Power Plant Example

In [1]:
from model import *
import pyomo.environ as pyo
from unit_models.pump import SVPump
from unit_models.turbine import SVTurbine
from unit_models.mixer import SVMixer
from unit_models.heat_exchanger import SVHeatExchanger
from idaes.core import FlowsheetBlock
from idaes.models.properties import iapws95
from idaes.core.util.model_statistics import degrees_of_freedom
from property_packages.build_package import build_package
from pyomo.network import Arc, SequentialDecomposition
import idaes.logger as idaeslog

m = pyo.ConcreteModel()
m.fs = FlowsheetBlock(dynamic=False)
m.fs.water = iapws95.Iapws95ParameterBlock()
m.fs.butane = build_package("helmholtz",["n-butane"],["Liq","Vap"])

# Add unit models
m.fs.preheater = SVHeatExchanger(hot_side={
    "property_package": m.fs.water,
    "has_pressure_change": False
}, cold_side={
    "property_package": m.fs.butane,
    "has_pressure_change": False
})
m.fs.vaporizer = SVHeatExchanger(hot_side={
    "property_package": m.fs.water,
    "has_pressure_change": False
}, cold_side={
    "property_package": m.fs.butane,
    "has_pressure_change": False
})
m.fs.brine_mixer = SVMixer(property_package=m.fs.water)

m.fs.turbine = SVTurbine(property_package=m.fs.butane)

m.fs.recuperator = SVHeatExchanger(hot_side={
    "property_package": m.fs.butane,
    "has_pressure_change": False
}, cold_side={
    "property_package": m.fs.butane,
    "has_pressure_change": False
})

m.fs.condenser = SVHeatExchanger(hot_side={
    "property_package": m.fs.butane,
    "has_pressure_change": False
}, cold_side={
    "property_package": m.fs.water,
    "has_pressure_change": False
})

m.fs.pump = SVPump(property_package=m.fs.butane)

# Connect up the arcs
m.fs.turbine_recuperator = Arc(source=m.fs.turbine.outlet, destination=m.fs.recuperator.hot_side_inlet)
m.fs.recuperator_condenser = Arc(source=m.fs.recuperator.hot_side_outlet, destination=m.fs.condenser.hot_side_inlet)
m.fs.condenser_pump = Arc(source=m.fs.condenser.hot_side_outlet, destination=m.fs.pump.inlet)
m.fs.pump_recuperator = Arc(source=m.fs.pump.outlet, destination=m.fs.recuperator.cold_side_inlet)
m.fs.recuperator_preheater = Arc(source=m.fs.recuperator.cold_side_outlet, destination=m.fs.preheater.cold_side_inlet)
m.fs.preheater_vaporizer = Arc(source=m.fs.preheater.cold_side_outlet, destination=m.fs.vaporizer.cold_side_inlet)
m.fs.vaporizer_turbine = Arc(source=m.fs.vaporizer.cold_side_outlet, destination=m.fs.turbine.inlet)

m.fs.vaporiser_mixer = Arc(source=m.fs.vaporizer.hot_side_outlet, destination=m.fs.brine_mixer.inlet_1)
m.fs.mixer_condenser = Arc(source=m.fs.brine_mixer.outlet, destination=m.fs.preheater.hot_side_inlet)

pyo.TransformationFactory("network.expand_arcs").apply_to(m)

register_inlet_ports(m.fs)

# Replace state variables
replace_state_var(m.fs.vaporizer.overall_heat_transfer_coefficient, m.fs.vaporizer.cold_side_outlet.enth_mol)
replace_state_var(m.fs.preheater.overall_heat_transfer_coefficient, m.fs.preheater.cold_side_outlet.enth_mol)
replace_state_var(m.fs.recuperator.overall_heat_transfer_coefficient, m.fs.recuperator.cold_side_outlet.enth_mol)

replace_state_var(m.fs.condenser.overall_heat_transfer_coefficient, m.fs.condenser.hot_side_outlet.enth_mol)

replace_state_var(m.fs.pump.work_mechanical, m.fs.pump.outlet.pressure)
replace_state_var(m.fs.turbine.work_mechanical, m.fs.turbine.outlet.pressure)



pprint_replacements(m.fs)

assert degrees_of_freedom(m) == 0

Replacements in block fs:
(Variable -> Replaced State Var)
  fs.preheater._enth_mol_cold_side_outlet_ref -> fs.preheater.overall_heat_transfer_coefficient
  fs.vaporizer._enth_mol_cold_side_outlet_ref -> fs.vaporizer.overall_heat_transfer_coefficient
  fs.turbine._pressure_outlet_ref -> fs.turbine.work_mechanical
  fs.recuperator._enth_mol_cold_side_outlet_ref -> fs.recuperator.overall_heat_transfer_coefficient
  fs.condenser._enth_mol_hot_side_outlet_ref -> fs.condenser.overall_heat_transfer_coefficient
  fs.pump._pressure_outlet_ref -> fs.pump.work_mechanical

Unreplaced state variables in block fs:
  fs.preheater.area
  fs.vaporizer.area
  fs.vaporizer._flow_mol_hot_side_inlet_ref
  fs.vaporizer._enth_mol_hot_side_inlet_ref
  fs.vaporizer._pressure_hot_side_inlet_ref
  fs.brine_mixer._flow_mol_inlet_2_ref
  fs.brine_mixer._enth_mol_inlet_2_ref
  fs.brine_mixer._pressure_inlet_2_ref
  fs.turbine.efficiency_isentropic
  fs.recuperator.area
  fs.condenser.area
  fs.condenser._flow_mol_

In [2]:
# Fix the variables to values we want
fs = m.fs

fs.preheater._enth_mol_cold_side_outlet_ref.fix(fs.butane.htpx(p=2300e3*pyo.units.Pa,T=(80 + 273.15)* pyo.units.K) )
fs.vaporizer._enth_mol_cold_side_outlet_ref.fix(fs.butane.htpx(p=2300e3*pyo.units.Pa,T=(200 + 273.15)* pyo.units.K))
fs.turbine._pressure_outlet_ref.fix(100e3)
fs.recuperator._enth_mol_cold_side_outlet_ref.fix(fs.butane.htpx(p=2300e3*pyo.units.Pa,T=(50 + 273.15)* pyo.units.K))
fs.condenser._enth_mol_hot_side_outlet_ref.fix(fs.butane.htpx(p=100e3*pyo.units.Pa,T=(20 + 273.15)* pyo.units.K))
fs.pump._pressure_outlet_ref.fix(2300e3)

fs.preheater.area.fix(5)
fs.vaporizer.area.fix(5)
fs.vaporizer._flow_mol_hot_side_inlet_ref.fix(1200)
fs.vaporizer._enth_mol_hot_side_inlet_ref.fix(fs.water.htpx(p=100e3*pyo.units.Pa, T=(240 + 273.15)* pyo.units.K))
fs.vaporizer._pressure_hot_side_inlet_ref.fix(100e3)
fs.brine_mixer._flow_mol_inlet_2_ref.fix(400)
fs.brine_mixer._enth_mol_inlet_2_ref.fix(fs.water.htpx(p=100e3* pyo.units.Pa,T=(40+ 273.15)* pyo.units.K))
fs.brine_mixer._pressure_inlet_2_ref.fix(150e3)
fs.turbine.efficiency_isentropic.fix(0.85)
fs.recuperator.area.fix(5)
fs.condenser.area.fix(5)
fs.condenser._flow_mol_cold_side_inlet_ref.fix(800)
fs.condenser._enth_mol_cold_side_inlet_ref.fix(fs.water.htpx(p=100e3*pyo.units.Pa,T=(5 + 273.15)* pyo.units.K))
fs.condenser._pressure_cold_side_inlet_ref.fix(100e3)
fs.pump.efficiency_pump.fix(0.85)

assert degrees_of_freedom(m) == 0

In [3]:
# Initialize the model with sequential decomposition


seq = SequentialDecomposition(iterLim=1,run_first_pass=True)
seq.set_tear_set([
     m.fs.condenser_pump,
     m.fs.recuperator_preheater,
     m.fs.vaporiser_mixer,
])

# Using the SD tool


seq.set_guesses_for(m.fs.pump.inlet, {
        "flow_mol":{0:1},
        "enth_mol": {0: -25208},
        "pressure": {0: 100e3}}) 
seq.set_guesses_for(m.fs.brine_mixer.inlet_1, {
        "flow_mol":{0:1200},
        "enth_mol": {0: 15000},
        "pressure": {0: 2300e3}}) 
seq.set_guesses_for(m.fs.preheater.hot_side_inlet,{
        "flow_mol":{0:1},
        "enth_mol": {0: -16414},
        "pressure": {0: 2300e3}}) 
G = seq.create_graph(m)


order = seq.calculation_order(G)
seq_blocks = []
for o in order:
        seq_blocks.append(o[0])
print("Order of initialisation:", [blk.name for blk in seq_blocks])


# We shouldn't need to check if a unit has already been initialised
# but because we have run_first_pass=True, we it runs twice.
# However, it fails on the second pass if we try to initialise again, potentially because
# our guesses are bad. Better to let ipopt handle it.
initialised = []

def function(unit):
        if unit.name not in initialised:
                print("Initialising Unit: ", unit.name)
                unit.initialize()
                initialised.append(unit.name)
        else:
                print("Skipping Unit (already initialised): ", unit.name)

res = seq.run(m, function)


Order of initialisation: ['fs.pump', 'fs.preheater', 'fs.vaporizer', 'fs.turbine', 'fs.recuperator', 'fs.condenser']
Initialising Unit:  fs.pump
2025-11-11 15:06:05 [INFO] idaes.init.fs.pump.control_volume: Initialization Complete
2025-11-11 15:06:05 [INFO] idaes.init.fs.pump: Initialization Complete: optimal - Optimal Solution Found
Initialising Unit:  fs.brine_mixer
2025-11-11 15:06:05 [INFO] idaes.init.fs.brine_mixer: Initialization Complete: optimal - Optimal Solution Found
Initialising Unit:  fs.preheater
2025-11-11 15:06:05 [INFO] idaes.init.fs.preheater.hot_side: Initialization Complete
2025-11-11 15:06:05 [INFO] idaes.init.fs.preheater.cold_side: Initialization Complete
2025-11-11 15:06:05 [INFO] idaes.init.fs.preheater: Initialization Completed, optimal - Optimal Solution Found
Initialising Unit:  fs.vaporizer
2025-11-11 15:06:05 [INFO] idaes.init.fs.vaporizer.hot_side: Initialization Complete
2025-11-11 15:06:05 [INFO] idaes.init.fs.vaporizer.cold_side: Initialization Complet

In [5]:
s = pyo.SolverFactory('ipopt')
results = s.solve(m, tee=True)
pyo.assert_optimal_termination(results)

Ipopt 3.13.2: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowledgement:
        HSL, a collection of Fortran codes for large-scale scientific
        computation. See http://